In [227]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

In [228]:
print(torch.cuda.is_available())
device = 'cuda'

True


In [229]:
transform = transforms.ToTensor()

training = datasets.MNIST('./data', train=True, download=True, transform=transform)
testing = datasets.MNIST('./data', train=False, download=True, transform=transform)

In [230]:
from torch.utils.data import random_split

train_set, dev_set = random_split(training, [50000, 10000])

In [231]:
training_loader = DataLoader(train_set, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size=32)
testing_loader = DataLoader(testing, batch_size=32)

In [232]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super().__init__()
        self.layers = [input_size] + hidden_layers + [output_size]
        self.network_layers = nn.ModuleList()
        self.relu = nn.ReLU()

        for i in range(len(self.layers)-1):
            self.network_layers.append(nn.Linear(self.layers[i], self.layers[i+1]))

    def forward(self, x):
        for i in range(len(self.network_layers)-1): # all layers except output
            x = self.network_layers[i](x)
            x = self.relu(x)
        #output
        x = self.network_layers[-1](x)
        return x

In [233]:
net = NeuralNetwork(784, [256], 10).cuda()
print(net.layers)

[784, 256, 10]


In [234]:
def dataset_accuracy(model, loader):
    model.eval()
    with torch.no_grad():
        correct_predictions = 0
        for X, y in loader:

            X = X.reshape(-1, 784).cuda()
            y = y.cuda()

            output = model(X)
            correct_predictions += torch.sum(torch.max(output, 1)[1] == y).item()
        model.train()
        return correct_predictions / len(loader.dataset)

In [235]:
import torch.optim

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)


epochs = 10
for epoch in range(epochs):

    epoch_loss = 0
    correct_predictions = 0

    for X, y in training_loader:

        X = X.reshape(-1, 784).cuda()
        y = y.cuda()

        optimizer.zero_grad()
        output = net(X)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        correct_predictions += torch.sum(torch.max(output, dim=1)[1] == y).item()

    loss_score = epoch_loss/len(training_loader)
    training_accuracy = correct_predictions/len(training_loader.dataset)
    dev_accuracy = dataset_accuracy(net, dev_loader)

    print(f"epoch: {epoch+1}  loss = {loss_score}, training accuracy = {training_accuracy}, dev accuracy = {dev_accuracy}")

print('------------------')
print(f'Test set accuracy = {dataset_accuracy(net, testing_loader)}')

epoch: 1  loss = 0.2699259196986669, training accuracy = 0.9236, dev accuracy = 0.9607
epoch: 2  loss = 0.11318894958311855, training accuracy = 0.96604, dev accuracy = 0.9706
epoch: 3  loss = 0.07408975253864213, training accuracy = 0.97764, dev accuracy = 0.9753
epoch: 4  loss = 0.053757797815611734, training accuracy = 0.98426, dev accuracy = 0.9755
epoch: 5  loss = 0.04065246758555757, training accuracy = 0.98724, dev accuracy = 0.9759
epoch: 6  loss = 0.031359427125385045, training accuracy = 0.99006, dev accuracy = 0.9791
epoch: 7  loss = 0.02416669944869098, training accuracy = 0.9927, dev accuracy = 0.9742
epoch: 8  loss = 0.019379476044814953, training accuracy = 0.9945, dev accuracy = 0.9774
epoch: 9  loss = 0.017476896469812493, training accuracy = 0.99498, dev accuracy = 0.9741
epoch: 10  loss = 0.013385888208829065, training accuracy = 0.99588, dev accuracy = 0.9752
------------------
Test set accuracy = 0.9752


In [236]:
# import matplotlib.pyplot as plt
# import numpy as np

# def visualize_mnist(loader, num_images = 5):

#     random_choices = np.random.randint(1, 60000, size=num_images)

#     features_list = []
#     labels_list = []

#     for choice in random_choices:
#         X, y = loader[choice]
#         X = X.reshape(-1, 784)

#         features_list.append(X)
#         labels_list.append(y)

#     fig, axes = plt.subplots(2, num_images, figsize=(20, 6))
#     for i in range(num_images):
#         axes[0, i].imshow(X, cmap='gray')

# visualize_mnist(testing_loader)